<a href="https://colab.research.google.com/github/caioitalo/projetos_estudos/blob/main/Web_scrapping_Selenium_Beautifulsoup4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alteração da versão ubuntu para funcionar com o colab

> Por conta de uma atualização na versão ubuntu utilizado com o colab, teve-se que utilizar um código após a instalação da biblioteca selenium para que este funcione nessa plataforma.

In [1]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 3.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.PX7QGP68Gc/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.BqhNcVBZvN/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.xy5SnQvr4e/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


# Instalação e importação de bibliotecas

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import json

In [4]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu focal InRelease
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:12 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Get:13 http://deb.debian.org/debian buster-upd

# Criação de webdriver e do Soup

## criação de webdriver selenium

In [18]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome( options=options)
    return driver

In [19]:
driver = web_driver()

In [20]:
# teste se o webdriver do selenium funciona
driver.get('https://www.google.com')
driver.title

'Google'

# Início Web-scrapping

In [21]:
driver.get('http://books.toscrape.com/catalogue/category/books/travel_2/index.html')
pag_atual = driver.current_url
acesso_bs = requests.get(pag_atual)
books_soup = BeautifulSoup(acesso_bs.text, 'html.parser')
catalogo = []

In [13]:
# coleta de links das páginas de outras categorias
links = []
for i in range(1, 51): 
  links.append(driver.find_element(By.CSS_SELECTOR,f'#default > div > div > div > aside > div.side_categories > ul > li > ul > li:nth-child({i}) > a').get_attribute('href'))


In [49]:
# coleta de livros de várias paginas
catalogo= []

i=-1
while True:
    livros = books_soup.find_all("article")
    for livro in livros:
      a = {'nome':livro.h3.a['title'],
          'categoria':books_soup.h1.get_text(),
          'preco':livro.find('p',class_='price_color').get_text()[2:],
          'rating':livro.p['class'][1],
          'estoque':livro.find('p',class_='instock availability').get_text().strip()}
      catalogo.append(a)
      a = None
    
    try:
      driver.find_element(By.CSS_SELECTOR, '#default > div > div > div > div > section > div:nth-child(2) > div > ul > li.next > a').click()
      driver.get(driver.current_url)
      acesso_bs = requests.get(driver.current_url)
      books_soup = BeautifulSoup(acesso_bs.text, 'html.parser')
      books_soup.find_all("article")
      print(f'mudou a pagina {driver.current_url}')
        
    except NoSuchElementException:
      i+=1
      print(i)
      if i == 50:
        break
      driver.get(links[i])
      acesso_bs = requests.get(links[i])
      books_soup = BeautifulSoup(acesso_bs.text, 'html.parser')
      books_soup.find_all("article")
      print(f'mudou a categoria {driver.current_url}')
      

0
mudou a categoria https://books.toscrape.com/catalogue/category/books/travel_2/index.html
1
mudou a categoria https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
mudou a pagina https://books.toscrape.com/catalogue/category/books/mystery_3/page-2.html
2
mudou a categoria https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
mudou a pagina https://books.toscrape.com/catalogue/category/books/historical-fiction_4/page-2.html
3
mudou a categoria https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html
mudou a pagina https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-2.html
mudou a pagina https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-3.html
mudou a pagina https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-4.html
4
mudou a categoria https://books.toscrape.com/catalogue/category/books/classics_6/index.html
5
mudou a categoria https://books.toscra

In [50]:
len(catalogo)

1001

In [38]:
catalogo

[{'nome': 'The Long Shadow of Small Ghosts: Murder and Memory in an American City',
  'categoria': 'Crime',
  'preco': '10.97',
  'rating': 'One',
  'estoque': 'In stock'},
 {'nome': 'Sharp Objects',
  'categoria': 'Mystery',
  'preco': '47.82',
  'rating': 'Four',
  'estoque': 'In stock'},
 {'nome': 'In a Dark, Dark Wood',
  'categoria': 'Mystery',
  'preco': '19.63',
  'rating': 'One',
  'estoque': 'In stock'},
 {'nome': 'The Past Never Ends',
  'categoria': 'Mystery',
  'preco': '56.50',
  'rating': 'Four',
  'estoque': 'In stock'},
 {'nome': 'A Murder in Time',
  'categoria': 'Mystery',
  'preco': '16.64',
  'rating': 'One',
  'estoque': 'In stock'},
 {'nome': 'The Murder of Roger Ackroyd (Hercule Poirot #4)',
  'categoria': 'Mystery',
  'preco': '44.10',
  'rating': 'Four',
  'estoque': 'In stock'},
 {'nome': 'The Last Mile (Amos Decker #2)',
  'categoria': 'Mystery',
  'preco': '54.21',
  'rating': 'Two',
  'estoque': 'In stock'},
 {'nome': 'That Darkness (Gardiner and Renner #1)

In [51]:
catalogo[-1]

{'nome': 'The Long Shadow of Small Ghosts: Murder and Memory in an American City',
 'categoria': 'Crime',
 'preco': '10.97',
 'rating': 'One',
 'estoque': 'In stock'}

In [52]:
objeto = json.dumps(catalogo,indent=5)
objeto

'[\n     {\n          "nome": "The Long Shadow of Small Ghosts: Murder and Memory in an American City",\n          "categoria": "Crime",\n          "preco": "10.97",\n          "rating": "One",\n          "estoque": "In stock"\n     },\n     {\n          "nome": "It\'s Only the Himalayas",\n          "categoria": "Travel",\n          "preco": "45.17",\n          "rating": "Two",\n          "estoque": "In stock"\n     },\n     {\n          "nome": "Full Moon over Noah\\u00e2\\u0080\\u0099s Ark: An Odyssey to Mount Ararat and Beyond",\n          "categoria": "Travel",\n          "preco": "49.43",\n          "rating": "Four",\n          "estoque": "In stock"\n     },\n     {\n          "nome": "See America: A Celebration of Our National Parks & Treasured Sites",\n          "categoria": "Travel",\n          "preco": "48.87",\n          "rating": "Three",\n          "estoque": "In stock"\n     },\n     {\n          "nome": "Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel

In [56]:
df = pd.read_json(objeto)
df

,nome,categoria,preco,rating,estoque
0,The Long Shadow of Small Ghosts: Murder and Me...,Crime,10.97,One,In stock
1,It's Only the Himalayas,Travel,45.17,Two,In stock
2,Full Moon over Noahâs Ark: An Odyssey to Mou...,Travel,49.43,Four,In stock
3,See America: A Celebration of Our National Par...,Travel,48.87,Three,In stock
4,Vagabonding: An Uncommon Guide to the Art of L...,Travel,36.94,Two,In stock
...,...,...,...,...,...
996,Why the Right Went Wrong: Conservatism--From G...,Politics,52.65,Four,In stock
997,Equal Is Unfair: America's Misguided Fight Aga...,Politics,56.86,One,In stock
998,Amid the Chaos,Cultural,36.58,One,In stock
999,Dark Notes,Erotica,19.19,Five,In stock


In [54]:
df = df.drop_duplicates()

In [55]:
df.groupby('categoria').size()

categoria
Academic                1
Add a comment          67
Adult Fiction           1
Art                     8
Autobiography           9
Biography               5
Business               12
Childrens              29
Christian               3
Christian Fiction       6
Classics               19
Contemporary            3
Crime                   1
Cultural                1
Default               152
Erotica                 1
Fantasy                48
Fiction                65
Food and Drink         30
Health                  4
Historical              2
Historical Fiction     26
History                18
Horror                 17
Humor                  10
Music                  13
Mystery                32
New Adult               6
Nonfiction            110
Novels                  1
Paranormal              1
Parenting               1
Philosophy             11
Poetry                 19
Politics                3
Psychology              7
Religion                7
Romance                35
Sc